In [1]:
# 파일 경로 찾아주는 함수 Path
from pathlib import Path

# 타입 힌트 처리용 모듈
from __future__ import annotations

# 그냥 os
import os

# math.log, math.exp 등의 함수 사용하기 위한 모듈
import math

# 데이터 전처리 및 분석용 모듈
import numpy as np
import pandas as pd

# wavelet
import pywt

# PCA
from sklearn.decomposition import PCA

In [ ]:
# -------------------------------------------------------------------------------------------------
# 예제 데이터 전처리 과정
# -------------------------------------------------------------------------------------------------

n = 100
err_sd = 30

# base_dir : py 파일이 저장된 폴더
base_dir = Path("data_250418").resolve().parent

# data_dir : 예제 데이터가 저장된 폴더
data_dir = base_dir / "data_250418"

CSV_FILES = [
    "000_Et_H_CO_n.csv",
    "002_Et_H_CO_H.csv",
    "008_Et_H_CO_L.csv",
    "028_Et_H_CO_M.csv",
]

originals: list[np.ndarray] = [] # 타입힌트 list[np.ndarray]
for fname in CSV_FILES:
    path = data_dir / fname

    # pandas data.frame
    df = pd.read_csv(path, header=None)

    # to.numpy()
    originals.append(df.iloc[:, 3:].to_numpy())

# N(0, err_sd^2) noise 생성
# rng = np.random.default_rng(seed= 123)
noisy = []
for ary in originals:
    rows, cols = ary.shape
    arr = np.empty((rows, cols, n))
    for i in range(n):
        rng = np.random.default_rng(seed= i+1)
        arr[..., i] = ary + rng.normal(scale=err_sd, size=(rows, cols))
    noisy.append(arr)

"""
noisy: list[np.ndarray] = [shape(2970, 8, 100), shape(2970, 8, 100), 
                           shape(2970, 8, 100), shape(2970, 8, 100)]
(noisy는 총 4개의 다차원 array를 담고 있는 list임)
해당 형식을 shape(2970, 8, 400)으로 하나의 array 형태로 재편집해야 함
single_array_data : shape (T, sensors, 4*n)
"""
rows, cols, n = noisy[0].shape

# single_array_data = data_origin
single_array_data = np.empty((rows, cols, 4 * n))
for k, ary in enumerate(noisy):
    single_array_data[..., k * n : (k + 1) * n] = ary

In [ ]:
# -------------------------------------------------------------------------------------------------
# 분류기 학습 과정
# -------------------------------------------------------------------------------------------------

# T_original = 2970 (시계열)
# n_sensors = 8 (변수의 수)
# n_samples = 400
# T = 2^12
T_original, n_sensors, n_samples = single_array_data.shape
T = 1 << (T_original - 1).bit_length() # T = 2^12
pad = T - T_original
w_per_sensor = T_original

# R 코드에서 filter.number = 2, family = "DaubExPhase"에 대응한다.
wavelet: str = "db2"
wavelet_features = np.zeros((n_samples, n_sensors * w_per_sensor))

for i in range(n_samples):
    for j in range(n_sensors):
        series = single_array_data[:, j, i]
        if pad:
            # 뒤를 0으로 패딩
            series = np.pad(series, (0, pad))

        # 웨이블렛 분해 결과 리스트
        coeff_lists = pywt.wavedec(series, wavelet=wavelet, mode="periodization")

        # 모든 레벨의 계수 + 근사 계수
        coeffs = np.concatenate(coeff_lists[1:] + [coeff_lists[0]])

        # 상위 w_per_sensor 개 계수 선택 (절대값 기준)
        # np.argsort = 오름차순으로 정렬되는 인덱스 배열 반환
        # [::-1] = 역순
        # [:w_per_sensor] = 상위 w_per_sensor개 만큼 자르기
        idx = np.argsort(np.abs(coeffs))[::-1][:w_per_sensor]
        sel = coeffs[idx]

        # w_per_sensor 단위로 대입
        wavelet_features[i, j * w_per_sensor : (j + 1) * w_per_sensor] = sel


# PCA 객체 인스턴스 생성
pca = PCA()

# PCA 적합 + PCA로 데이터 축약 과정
wavelet_features_reduced = pca.fit_transform(wavelet_features)
var_threshold: float = 0.80

# 누적 분산
cum_var = np.cumsum(pca.explained_variance_ratio_)

# 누적 분산이 처음 var_threshold(기본값) 넘는 인덱스
n_pcs = int(np.searchsorted(cum_var, var_threshold) + 1)
print(f"Number of PCs explaining >= {var_threshold*100:.0f}% variance: {n_pcs}")

# 데이터 축약
# wavelet_features_reduced : shape(400, 4)
wavelet_features_reduced = wavelet_features_reduced[:, :n_pcs]

# 분류기 학습 및 저장
# res : 통계량을 저장한 dictionary
res: dict[str, np.ndarray] = {}

# np.eye : 주어진 크기만큼 대각행렬을 생성한다.
regularisation = np.eye(n_pcs) * 1e-4

# 분류기 학습(naive bayes)
for k in range(4):
    # cls : shape (100, 4)
    cls = wavelet_features_reduced[k * n : (k + 1) * n]
    res[f"mean{k+1}"] = cls.mean(axis=0)
    # np.cov : 공분산 행렬 구하는 함수
    # rowvar = FALSE : 열을 변수로 보고 행을 관측치로 보겠다는 뜻
    # singular problem 문제를 해소하기 위해 작은 regularisation 대각행렬 더함
    res[f"cov{k+1}"] = np.cov(cls, rowvar=False) + regularisation

Number of PCs explaining >= 80% variance: 3


In [ ]:
# -------------------------------------------------------------------------------------------------
# (번외) r에서 만든 data_original 데이터 가져오기
# -------------------------------------------------------------------------------------------------
"""데이터의 구조가 (2900, 3200)이며 sample=1의 센서 1~8열, sample=2의 센서 1~8열, ...
sample=400의 센서 1~8열 형태로 되어 있기 때문에 일반적인 reshape를 사용할 수 없다.
reshape를 할 경우, 열에 대해서 센서 1의 400개 sample, 센서 2의 400개 sample, ... 
이런 식으로 인식하고 처리하기 때문이다. 
따라서 transpose로 센서와 sample의 순서를 바꿔야한다."""

single_array_data = pd.read_csv(data_dir / "r.csv", header=None)
single_array_data = single_array_data.iloc[1:, 1:].to_numpy(copy=False) # 첫번째 열과 첫번째 행 지우기(제목 열, 제목 행)

# (2970, 400, 8) 형태로 reshape
single_array_data = single_array_data.reshape(single_array_data.shape[0], 400, 8)

# 센서 축을 가운데로 이동시키며 (2970, 8, 400) 형태로 바꾸기
single_array_data = single_array_data.transpose(0, 2, 1)

C:\Users\rkdal\AppData\Local\Temp\ipykernel_12756\1542954946.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,25